In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [ ]:
dataset_train= pd.read_csv('Google_Stock_Train.csv')

In [ ]:
dataset_train.head(10)

In [ ]:
training_set= dataset_train.iloc[:, 1:2].values

In [ ]:
training_set

In [ ]:
training_set.shape

In [ ]:
plt.figure(figsize=(18,8))
plt.plot(dataset_train['Open'])
plt.title("Google Stock Open Prices")
plt.xlabel("Time(oldest->latest)")
plt.ylabel("Stock Open Price")
plt.show()

In [ ]:
sc= MinMaxScaler(feature_range=(0,1))
training_set_scaled= sc.fit_transform(training_set)

In [ ]:
X_train= []
y_train= []
for i in range(60, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-60: i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train= np.array(X_train), np.array(y_train)

In [ ]:
X_train[0]

In [ ]:
y_train[0]

In [ ]:
X_train= np.reshape(X_train, newshape=(X_train.shape[0], X_train.shape[1], 1))

In [ ]:
X_train.shape

In [ ]:
from tensorflow.keras.layers import Input

regressor = Sequential()
regressor.add(Input(shape=(X_train.shape[1], 1)))  # Input layer
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(rate=0.2))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(rate=0.2))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(rate=0.2))

regressor.add(LSTM(units=50, return_sequences=False))
regressor.add(Dropout(rate=0.2))

regressor.add(Dense(units=1))

In [ ]:
regressor.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
regressor.fit(x=X_train, y=y_train, batch_size=32, epochs=100)

101/101 ━━━━━━━━━━━━━━━━━━━━ 12s 117ms/step - loss: 0.0011
Epoch 22/100
  1/101 ━━━━━━━━━━━━━━━━━━━━ 18s 187ms/step - loss: 7.9121e-04

In [ ]:
regressor.summary()

In [ ]:
dataset_test= pd.read_csv('Google_Stock_Test.csv')

In [ ]:
dataset_test.head()

In [ ]:
dataset_total= pd.concat((dataset_train['Open'], dataset_test['Open']), axis= 0)
inputs= dataset_total[len(dataset_total)-len(dataset_test) -60:].values

In [ ]:
inputs= inputs.reshape(-1,1)
inputs=sc.transform(inputs)

In [ ]:
len(inputs)

In [ ]:
X_test= []
for i in range(60, len(inputs)):
    X_test.append(inputs[i-60:i, 0])
X_test= np.array(X_test)
X_test= np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
X_test.shape

In [ ]:
predicted_stock_price= regressor.predict(X_test)

In [ ]:
predicted_stock_price= sc.inverse_transform(predicted_stock_price)

In [ ]:
plt.plot(real_stock_price, color='red', label='Real price')
plt.plot(predicted_stock_price, color='blue', label='Predicted price')
plt.title('Google Price Prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()

# 🧠 How the Code Uses RNN

Even though the code does not directly mention "RNN", it **uses Recurrent Neural Network (RNN) concepts** by implementing **LSTM layers**, which are an advanced type of RNN.

---

## 🔄 What is an RNN?

**Recurrent Neural Networks (RNNs)** are designed to handle **sequential data**, such as time series or natural language. Unlike traditional neural networks, RNNs have **memory**—they pass information (hidden states) from one step to the next in the sequence, allowing them to learn **temporal dependencies**.

However, basic RNNs struggle with **long-term dependencies** due to the **vanishing/exploding gradient problem**.

---

## 🔒 What is LSTM (Long Short-Term Memory)?

**LSTM** is a special type of RNN that introduces:

- **Memory Cells**: Maintain information over long sequences.
- **Gates**: Control the flow of information (input gate, forget gate, output gate).

LSTMs effectively solve the vanishing gradient problem and are widely used in:

- Stock price prediction
- Language modeling
- Speech recognition
- Weather forecasting

---

## 🔍 How LSTM Implements RNN in the Code

Your model uses LSTM layers like this:

```python
regressor.add(LSTM(units=50, return_sequences=True))
```

This means:
- The model processes **sequences of 60 days** of stock prices.
- It **remembers previous steps** and learns from them.
- It models **temporal dependencies** using recurrent behavior.

The input is reshaped as:

```python
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
```

Which gives the shape `(samples, time steps, features)` — the required input for any RNN/LSTM-based architecture.

---

## 🏗️ Stacked LSTM = Deep RNN

You use **4 LSTM layers**, which means you're building a **deep RNN architecture** that can learn complex patterns at different levels:

```python
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(rate=0.2))
...
```

The output of one LSTM is passed to the next, allowing the model to capture both **short-term and long-term dependencies**.

---

## 📊 Conceptual Flow of the Model

```
[Price_t-60, ..., Price_t-1] 
       ↓
    LSTM Layer 1
       ↓
    LSTM Layer 2
       ↓
    LSTM Layer 3
       ↓
    LSTM Layer 4
       ↓
    Dense Layer (Output)
       ↓
 Predicted Price_t
```

---

## ✅ Summary: RNN in Action

| Feature                | RNN Concept               | Your Code Usage                           |
|------------------------|---------------------------|--------------------------------------------|
| Sequence modeling      | ✅                         | 60 time steps as input                     |
| Temporal memory        | ✅                         | LSTM layers remember previous steps        |
| Gradient issues fixed  | ❌ (Vanilla RNN) → ✅ (LSTM) | LSTM gates solve vanishing gradient        |
| Stacked architecture   | ✅                         | 4 LSTM layers form a deep RNN              |
| Prediction output      | ✅                         | Dense layer produces one stock price       |

---

### 📎 TL;DR

Your model **is an RNN**, implemented using **LSTM layers**, which provide better performance on time-series data by retaining long-term dependencies, controlling memory via gates, and avoiding training problems found in vanilla RNNs.

